In [1]:
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import itertools
import csv , codecs
import glob
from PIL import Image
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn. decomposition import KernelPCA
from sklearn. model_selection import train_test_split
from sklearn. preprocessing import StandardScaler

In [2]:
data_dir = "./EEG_data/"

## CSV 파일 생성 함수

In [3]:
def dataset_csv_3():
    normalize_power = str(input("Relative, Abs :"))
    excel_dir = "./EEG_data" + '/' + normalize_power
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    subject = int(input("subject :"))
    book = Workbook()
    sheet = book.active
    sheet.cell(row=1, column=161).value = 'target'
 
    # x축 생성   
    for i in range(1,33):
        sheet.cell(row=1, column=i).value = 'ACh' + str(i)
        sheet.cell(row=1, column=i+32).value = 'BCh' + str(i)
        sheet.cell(row=1, column=i+64).value = 'DCh' + str(i)
        sheet.cell(row=1, column=i+96).value = 'GCh' + str(i)
        sheet.cell(row=1, column=i+128).value = 'TCh' + str(i)
        
    # 파일을 순차적으로 열어서 셀 영역을 복사
    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=20*k+l,column=32*j+m-1).value = source.cell(row=l,column=m).value
        

    with open('./dataset_eeg_3.csv', 'w', newline="") as f:
        c = csv.writer(f)
        for r in sheet.rows:
            c.writerow([cell.value for cell in r])
            
    df = pd.read_csv('./dataset_eeg_3.csv')
    df[df =='                      NaN'] =np.nan
    df = df.fillna(0.0).astype('float64')
    pd.options.mode.chained_assignment = None
#1-back mouse, nose, o2
    df["target"][0*subject:1*subject] = "0" #0:20
    df["target"][1*subject:2*subject] = "1" #21:40
    df["target"][2*subject:3*subject] = "2" #41:60
#2-back mouse, nose, o2
    df["target"][3*subject:4*subject] = "0" #60:80
    df["target"][4*subject:5*subject] = "1" #80:100
    df["target"][5*subject:6*subject] = "2" #100:120
#rest close eye mouse, nose, o2
    df["target"][6*subject:7*subject] = "0" #120:140
    df["target"][7*subject:8*subject] = "1" #140:160
    df["target"][8*subject:9*subject] = "2" #160:180
#rest open eye mouse, nose, o2
    df["target"][9*subject:10*subject] = "0" #180:200
    df["target"][10*subject:11*subject] = "1" #200:220
    df["target"][11*subject:12*subject] = "2" #220:240
    df.astype('float64')
    df.to_csv('./dataset_eeg_3.csv')
    
    return df
            


In [4]:
eeg_data_3 = dataset_csv_3()

Relative, Abs : Relative
subject : 20


C:\Users\whbom\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [5]:
state={}

state["1back_mouse"]=eeg_data_3.iloc[0:20]
state["1back_nose"]=eeg_data_3.iloc[20:40]
state["1back_O2"]=eeg_data_3.iloc[40:60]

state["2back_mouse"]=eeg_data_3.iloc[60:80]
state["2back_nose"]=eeg_data_3.iloc[80:100]
state["2back_O2"]=eeg_data_3.iloc[100:120]

state["close_rest_mouse"]=eeg_data_3.iloc[120:140]
state["close_rest_nose"]=eeg_data_3.iloc[140:160]
state["close_rest_O2"]=eeg_data_3.iloc[160:180]

state["open_rest_mouse"]=eeg_data_3.iloc[180:200]
state["open_rest_nose"]=eeg_data_3.iloc[200:220]
state["open_rest_O2"]=eeg_data_3.iloc[220:240]


In [6]:
ds_train = {}
ds_test = {}


ds_train={"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
       
       
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
       "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
       "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])
       
       
      }

ds_test = {"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
        
        
               
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
        "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
        "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])

       }




In [7]:
X.shape

NameError: name 'X' is not defined

In [ ]:
Y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True) 
skf.get_n_splits(X,Y)



In [ ]:
for train_index, test_index in skf.split(X,Y):
    
    print(Y.shape)

In [ ]:
for train_index, test_index in skf.split(X,Y):

    
    kernel_linear_pca = KernelPCA(n_components=0.8, kernel="linear")
    print(X)
    
    if(mode == 'kernel_linear') : 
        kernel_linear_pca.fit(X,Y)
        X_final = kernel_linear_pca.transform(X)

In [7]:
def ST_PCA_RF_M_N_O2 (tasks, k_parameter, num_iter):


        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,'linear')
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-1])
            Y = np.array(train_3['target'])

            rnd_clf = RandomForestClassifier(n_estimators=1000,max_depth= None, n_jobs=-1,class_weight="balanced" )
#             rnd_clf = RandomForestClassifier(n_estimators=1000,max_features= 160,n_jobs=-1 )

            all_scores = []
            Mouse = 0
            Nose = 0
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)
                

                for train_index, test_index in skf.split(X,Y):

                    pca = PCA(n_components=0.8)
                    
                    pca.fit(X,Y)
                    X_final = pca.transform(X)
#                         X_final = X
                    

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                    test = np.array(test_3.iloc[:,0:-1])
                  
                    test = pca.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa=='0')
                    Nose = Nose+ np.count_nonzero(aaa=='1')

            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))

In [9]:
def Linear_KPCA_RF_M_N_O2 (tasks, k_parameter, num_iter):

        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,'kernel_linear'))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-1])
            Y = np.array(train_3['target'])
            

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1 )
#             rnd_clf = RandomForestClassifier(n_estimators=1000,max_features= 160,n_jobs=-1 )

            all_scores = []
            Mouse = 0
            Nose = 0
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)
                
                for train_index, test_index in skf.split(X,Y):
                    
                    X_final = X
                    kernel_linear_pca = KernelPCA(n_components=7, kernel="linear")
                    kernel_linear_pca.fit(X,Y)
                    X_final = kernel_linear_pca.transform(X)
#                         X_final = X
                    

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                
                
                    test = np.array(test_3.iloc[:,0:-1])

                    test = kernel_linear_pca.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa=='0')
                    Nose = Nose+ np.count_nonzero(aaa=='1')

            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))
            

In [15]:
Linear_KPCA_RF_M_N_O2(tasks5,k_parameter = 10,num_iter = 10)

-------------------
training:open_rest_m_n/test:open_rest_mouse/mode:kernel_linear
model_score:0.3625
mouse:1865(0.93), nose:135(0.068) / total:2000
-------------------
training:open_rest_m_n/test:close_rest_mouse/mode:kernel_linear
model_score:0.345
mouse:834(0.42), nose:1166(0.58) / total:2000
-------------------
training:open_rest_m_n/test:1back_mouse/mode:kernel_linear
model_score:0.385
mouse:1509(0.75), nose:491(0.25) / total:2000
-------------------
training:open_rest_m_n/test:2back_mouse/mode:kernel_linear
model_score:0.3725
mouse:1174(0.59), nose:826(0.41) / total:2000
-------------------
training:open_rest_m_n/test:open_rest_nose/mode:kernel_linear
model_score:0.3325
mouse:126(0.063), nose:1874(0.94) / total:2000
-------------------
training:open_rest_m_n/test:close_rest_nose/mode:kernel_linear
model_score:0.3575
mouse:748(0.37), nose:1252(0.63) / total:2000
-------------------
training:open_rest_m_n/test:1back_nose/mode:kernel_linear
model_score:0.3675
mouse:1311(0.66), nose:

## Task

In [10]:
tasks5 = [
    ('open_rest_m_n', 'open_rest_mouse'),
    ('open_rest_m_n','close_rest_mouse'),
    ('open_rest_m_n','1back_mouse'),
    ('open_rest_m_n','2back_mouse'),
    
    
    ('open_rest_m_n','open_rest_nose'),
    ('open_rest_m_n','close_rest_nose'),
    ('open_rest_m_n','1back_nose'),
    ('open_rest_m_n','2back_nose'),
]
print("Standard_PCA-RF")
ST_PCA_RF_M_N_O2(tasks5,k_parameter = 10,num_iter = 1)
print("------------------------------------------")
print("------------------------------------------")
print("------------------------------------------")
print("Lineaer_KPCA-RF")
Linear_KPCA_RF_M_N_O2(tasks5,k_parameter = 10,num_iter = 10)


Standard_PCA-RF
-------------------
training:open_rest_m_n/test:open_rest_mouse/mode:linear
model_score:0.175
mouse:182(0.91), nose:18(0.09) / total:200
-------------------
training:open_rest_m_n/test:close_rest_mouse/mode:linear
model_score:0.2
mouse:105(0.53), nose:95(0.47) / total:200
-------------------
training:open_rest_m_n/test:1back_mouse/mode:linear
model_score:0.225
mouse:116(0.58), nose:84(0.42) / total:200
-------------------
training:open_rest_m_n/test:2back_mouse/mode:linear
model_score:0.3
mouse:104(0.52), nose:96(0.48) / total:200
-------------------
training:open_rest_m_n/test:open_rest_nose/mode:linear
model_score:0.2
mouse:14(0.07), nose:186(0.93) / total:200
-------------------
training:open_rest_m_n/test:close_rest_nose/mode:linear
model_score:0.225
mouse:80(0.4), nose:120(0.6) / total:200
-------------------
training:open_rest_m_n/test:1back_nose/mode:linear
model_score:0.275
mouse:99(0.49), nose:101(0.51) / total:200
-------------------
training:open_rest_m_n/tes

KeyboardInterrupt: 

## csv 전환

In [21]:
def save_record (tasks, k_parameter, num_iter):
    result = []

#     for mode in ['no_lda','lda']:
    for mode in ['linear']:
        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-1])
            Y = np.array(train_3['target'])

            rnd_clf = RandomForestClassifier(n_estimators=1000,max_depth= None, n_jobs=-1,class_weight="balanced" )

            all_scores = []
            Mouse = 0
            Nose = 0
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y):

                    X_final = X
                    pca = PCA(n_components=0.8)
                    if(mode == 'linear') : 
                        pca.fit(X,Y)
#                         X_final = X
                        X_final = pca.transform(X)
                    

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                    test = np.array(test_3.iloc[:,0:-1])
                    if mode == 'linear':
#                         pass
                        test = pca.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa=='0')
                    Nose = Nose+ np.count_nonzero(aaa=='1')

            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose))) 
            
            
            
            
            result.append([ task_train, task_test,np.mean(all_scores) ,'{}'.format(Mouse),'{:.2}'.format(Mouse/(Mouse+Nose)),'{}'.format(Nose),'{:.2}'.format(Nose/(Mouse+Nose)),'{}'.format(Mouse+Nose) ])
            df_standard = pd.DataFrame(data=result, columns=['training','test','model_score', 'mouse_num','mouse_percent','nose_num','nose_percent','total'  ])
       
    
    for mode in ['kernel_linear']:
        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-1])
            Y = np.array(train_3['target'])
            

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1 )
#             rnd_clf = RandomForestClassifier(n_estimators=1000,max_features= 160,n_jobs=-1 )

            all_scores = []
            Mouse = 0
            Nose = 0
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)
                
                for train_index, test_index in skf.split(X,Y):
                    
                    X_final = X
                    kernel_linear_pca = KernelPCA(n_components=7, kernel="linear")
                    if(mode == 'kernel_linear'): 
                        kernel_linear_pca.fit(X,Y)
                        X_final = kernel_linear_pca.transform(X)
#                         X_final = X
                    

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                
                
                    test = np.array(test_3.iloc[:,0:-1])
                    if mode == 'kernel_linear':
#                         pass
                        test = kernel_linear_pca.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa=='0')
                    Nose = Nose+ np.count_nonzero(aaa=='1')

            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))
            
    df_standard.append([ task_train, task_test,np.mean(all_scores) ,'{}'.format(Mouse),'{:.2}'.format(Mouse/(Mouse+Nose)),'{}'.format(Nose),'{:.2}'.format(Nose/(Mouse+Nose)),'{}'.format(Mouse+Nose),pca_kernel ])
   
    return df_standard
    
    
            
            


            

In [22]:
task = [
    ('close_rest_m_n','open_rest_mouse'),
    ('close_rest_m_n','close_rest_mouse'),
     ('close_rest_m_n','1back_mouse'),
     ('close_rest_m_n','2back_mouse'),
    
    ('close_rest_m_n','open_rest_nose'),
    ('close_rest_m_n','close_rest_nose'),
     ('close_rest_m_n','1back_nose'),
     ('close_rest_m_n','2back_nose'),
    
    ('close_rest_m_n','open_rest_O2'),    
    ('close_rest_m_n','close_rest_O2'),
     ('close_rest_m_n','1back_O2'),
     ('close_rest_m_n','2back_O2'),
    
    
    
    
    
    ('open_rest_m_n', 'open_rest_mouse'),
    ('open_rest_m_n','close_rest_mouse'),
    ('open_rest_m_n','1back_mouse'),
    ('open_rest_m_n','2back_mouse'),
    
    
    ('open_rest_m_n','open_rest_nose'),
    ('open_rest_m_n','close_rest_nose'),
    ('open_rest_m_n','1back_nose'),
    ('open_rest_m_n','2back_nose'),
    
    ('open_rest_m_n', 'open_rest_O2'),
    ('open_rest_m_n','close_rest_O2'),
    ('open_rest_m_n','1back_O2'),
    ('open_rest_m_n','2back_O2'),
    
    ('1back_m_n', 'open_rest_mouse'),
    ('1back_m_n','close_rest_mouse'),
    ('1back_m_n','1back_mouse'),
    ('1back_m_n','2back_mouse'),
    
    
    ('1back_m_n','open_rest_nose'),
    ('1back_m_n','close_rest_nose'),
    ('1back_m_n','1back_nose'),
    ('1back_m_n','2back_nose'),
    
    ('1back_m_n', 'open_rest_O2'),
    ('1back_m_n','close_rest_O2'),
    ('1back_m_n','1back_O2'),
    ('1back_m_n','2back_O2'),
    
    ('2back_m_n', 'open_rest_mouse'),
    ('2back_m_n','close_rest_mouse'),
    ('2back_m_n','1back_mouse'),
    ('2back_m_n','2back_mouse'),
    
    
    ('2back_m_n','open_rest_nose'),
    ('2back_m_n','close_rest_nose'),
    ('2back_m_n','1back_nose'),
    ('2back_m_n','2back_nose'),
    
    ('2back_m_n', 'open_rest_O2'),
    ('2back_m_n','close_rest_O2'),
    ('2back_m_n','1back_O2'),
    ('2back_m_n','2back_O2'),

    ('total_m_n', 'open_rest_mouse'),
    ('total_m_n','close_rest_mouse'),
    ('total_m_n','1back_mouse'),
    ('total_m_n','2back_mouse'),
    
    
    ('total_m_n','open_rest_nose'),
    ('total_m_n','close_rest_nose'),
    ('total_m_n','1back_nose'),
    ('total_m_n','2back_nose'),
    
    ('total_m_n', 'open_rest_O2'),
    ('total_m_n','close_rest_O2'),
    ('total_m_n','1back_O2'),
    ('total_m_n','2back_O2'),
    
    

 ]




In [ ]:
print("Linear-RF")
save_linear = save_record(task,k_parameter = 10,num_iter = 10)
save_linear.to_csv('linear_result.csv',mode='w')


Linear-RF
-------------------
training:close_rest_m_n/test:open_rest_mouse/mode:linear


In [ ]:
save_linear